In [ ]:
from stock_data import fetch_single_stock_data
from stock_features import prepare_data_for_ml, create_target_variable
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Define the parameters for your data pipeline
tickers_list = ['PG', 'KO', 'PEP', 'WMT', 'COST', '^GSPC']
start_date_str = '2010-01-01'
end_date_str = '2023-01-01'
output_filename = "consumer_stocks_final_engineered.csv"

# Make the single function call to run the entire pipeline
final_engineered_df = prepare_data_for_ml(
    tickers=tickers_list,
    start_date=start_date_str,
    end_date=end_date_str,
    output_engineered_csv=output_filename
)

# 2. Now, create your target variable just before modeling
# You can easily iterate through different windows and thresholds here
ticker = 'WMT'

# Example 1: Next-day target
data_next_day_target = create_target_variable(final_engineered_df.copy(), ticker, window=1, threshold=0)

# Example 2: Weekly target with a 1% threshold
data_weekly_target = create_target_variable(final_engineered_df.copy(), ticker, window=5, threshold=0.01)

# Now, you would proceed with splitting and modeling using either
# `data_next_day_target` or `data_weekly_target`.
# This is a much better structure for experimenting.


--- Starting Data Preparation Pipeline ---
Fetching data for 6 tickers from 2010-01-01 to 2023-01-01...

Discovered stock prefixes: ['COST', 'KO', 'PEP', 'PG', 'WMT', '^GSPC']

Processing features for stock prefix: COST
  - Calculating ATR for COST (window=14)...
  - Calculating RSI for COST (window=14)...
  - Calculating MACD for COST (fast=12, slow=26, signal=9)...

Processing features for stock prefix: KO
  - Calculating ATR for KO (window=14)...
  - Calculating RSI for KO (window=14)...
  - Calculating MACD for KO (fast=12, slow=26, signal=9)...

Processing features for stock prefix: PEP
  - Calculating ATR for PEP (window=14)...
  - Calculating RSI for PEP (window=14)...
  - Calculating MACD for PEP (fast=12, slow=26, signal=9)...

Processing features for stock prefix: PG
  - Calculating ATR for PG (window=14)...
  - Calculating RSI for PG (window=14)...
  - Calculating MACD for PG (fast=12, slow=26, signal=9)...

Processing features for stock prefix: WMT
  - Calculating ATR for 

In [6]:
final_engineered_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3222 entries, 2010-03-16 to 2022-12-29
Columns: 296 entries, Close_COST to ^GSPC_Volume_MA_Ratio_lag5
dtypes: float64(290), int64(6)
memory usage: 7.3 MB
